# A Pythonic Card Deck

Let's code a package to represent decks of playing cards. First, we'll create a simple class to represent individual cards. It's so simple we just need two data attributes and no methods. Python has a factory to make such simple classes: ``collections.namedtuple``.

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

Note that the ``namedtuple`` factory takes two arguments: the name of the class to create and a sequence of attribute names.

The ``Card`` class can then be instantiated as usual.

In [2]:
beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

The string representation of a ``Card`` instance is so explicit that you can actually apply ``eval()`` to it and get a clone of the object described:

In [3]:
my_card = eval(repr(beer_card))
my_card == beer_card

True

We are now ready to code the class to represent a deck of cards. I'll call it `` FrenchDeck`` since that is the formal name of the 52 card deck with 4 suits used not only in France but also in most of the Western hemisphere.

In [4]:
class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

The code is short but you'll se it does a lot.

The trickiest part is the use of a list comprehension in the initializer to build a list of cards by doing a cartesian product of suits and ranks. The logic of that list comprehension is explained in chapter 2 of Fluent Python, but right now we just want to focus on the external behavior of the class, so please believe that ``self._cards`` holds a list of 52 ``Card`` instances, as you'll see right away.

In [5]:
deck = FrenchDeck()

In [6]:
len(deck)

52

In [7]:
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

In [8]:
deck[12::13]

[Card(rank='A', suit='spades'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='A', suit='hearts')]

In [9]:
Card('Q', 'hearts') in deck

True

In [10]:
Card('Z', 'clubs') in deck

False

In [11]:
for card in deck:
  print(card)

Card(rank='2', suit='spades')
Card(rank='3', suit='spades')
Card(rank='4', suit='spades')
Card(rank='5', suit='spades')
Card(rank='6', suit='spades')
Card(rank='7', suit='spades')
Card(rank='8', suit='spades')
Card(rank='9', suit='spades')
Card(rank='10', suit='spades')
Card(rank='J', suit='spades')
Card(rank='Q', suit='spades')
Card(rank='K', suit='spades')
Card(rank='A', suit='spades')
Card(rank='2', suit='diamonds')
Card(rank='3', suit='diamonds')
Card(rank='4', suit='diamonds')
Card(rank='5', suit='diamonds')
Card(rank='6', suit='diamonds')
Card(rank='7', suit='diamonds')
Card(rank='8', suit='diamonds')
Card(rank='9', suit='diamonds')
Card(rank='10', suit='diamonds')
Card(rank='J', suit='diamonds')
Card(rank='Q', suit='diamonds')
Card(rank='K', suit='diamonds')
Card(rank='A', suit='diamonds')
Card(rank='2', suit='clubs')
Card(rank='3', suit='clubs')
Card(rank='4', suit='clubs')
Card(rank='5', suit='clubs')
Card(rank='6', suit='clubs')
Card(rank='7', suit='clubs')
Card(rank='8', sui

In [12]:
for card in reversed(deck):
  print(card)

Card(rank='A', suit='hearts')
Card(rank='K', suit='hearts')
Card(rank='Q', suit='hearts')
Card(rank='J', suit='hearts')
Card(rank='10', suit='hearts')
Card(rank='9', suit='hearts')
Card(rank='8', suit='hearts')
Card(rank='7', suit='hearts')
Card(rank='6', suit='hearts')
Card(rank='5', suit='hearts')
Card(rank='4', suit='hearts')
Card(rank='3', suit='hearts')
Card(rank='2', suit='hearts')
Card(rank='A', suit='clubs')
Card(rank='K', suit='clubs')
Card(rank='Q', suit='clubs')
Card(rank='J', suit='clubs')
Card(rank='10', suit='clubs')
Card(rank='9', suit='clubs')
Card(rank='8', suit='clubs')
Card(rank='7', suit='clubs')
Card(rank='6', suit='clubs')
Card(rank='5', suit='clubs')
Card(rank='4', suit='clubs')
Card(rank='3', suit='clubs')
Card(rank='2', suit='clubs')
Card(rank='A', suit='diamonds')
Card(rank='K', suit='diamonds')
Card(rank='Q', suit='diamonds')
Card(rank='J', suit='diamonds')
Card(rank='10', suit='diamonds')
Card(rank='9', suit='diamonds')
Card(rank='8', suit='diamonds')
Card(r

In [13]:
for n, card in enumerate(deck, 1):
  print(n, card)

1 Card(rank='2', suit='spades')
2 Card(rank='3', suit='spades')
3 Card(rank='4', suit='spades')
4 Card(rank='5', suit='spades')
5 Card(rank='6', suit='spades')
6 Card(rank='7', suit='spades')
7 Card(rank='8', suit='spades')
8 Card(rank='9', suit='spades')
9 Card(rank='10', suit='spades')
10 Card(rank='J', suit='spades')
11 Card(rank='Q', suit='spades')
12 Card(rank='K', suit='spades')
13 Card(rank='A', suit='spades')
14 Card(rank='2', suit='diamonds')
15 Card(rank='3', suit='diamonds')
16 Card(rank='4', suit='diamonds')
17 Card(rank='5', suit='diamonds')
18 Card(rank='6', suit='diamonds')
19 Card(rank='7', suit='diamonds')
20 Card(rank='8', suit='diamonds')
21 Card(rank='9', suit='diamonds')
22 Card(rank='10', suit='diamonds')
23 Card(rank='J', suit='diamonds')
24 Card(rank='Q', suit='diamonds')
25 Card(rank='K', suit='diamonds')
26 Card(rank='A', suit='diamonds')
27 Card(rank='2', suit='clubs')
28 Card(rank='3', suit='clubs')
29 Card(rank='4', suit='clubs')
30 Card(rank='5', suit='clu

In [14]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

In [15]:
spades_high(Card('2', 'clubs')), spades_high(Card('A', 'spades'))

(0, 51)

In [16]:
for card in sorted(deck, key=spades_high):
     print(card)

Card(rank='2', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='3', suit='clubs')
Card(rank='3', suit='diamonds')
Card(rank='3', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='4', suit='clubs')
Card(rank='4', suit='diamonds')
Card(rank='4', suit='hearts')
Card(rank='4', suit='spades')
Card(rank='5', suit='clubs')
Card(rank='5', suit='diamonds')
Card(rank='5', suit='hearts')
Card(rank='5', suit='spades')
Card(rank='6', suit='clubs')
Card(rank='6', suit='diamonds')
Card(rank='6', suit='hearts')
Card(rank='6', suit='spades')
Card(rank='7', suit='clubs')
Card(rank='7', suit='diamonds')
Card(rank='7', suit='hearts')
Card(rank='7', suit='spades')
Card(rank='8', suit='clubs')
Card(rank='8', suit='diamonds')
Card(rank='8', suit='hearts')
Card(rank='8', suit='spades')
Card(rank='9', suit='clubs')
Card(rank='9', suit='diamonds')
Card(rank='9', suit='hearts')
Card(rank='9', suit='spades')
Card(rank='10', suit='clubs')
Ca